In [1]:
import os
import gc
import logging
import time

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
from sklearn.metrics import mean_squared_error, roc_auc_score, roc_curve, accuracy_score, auc
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn import svm
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.base import clone
from sklearn.pipeline import Pipeline

from keras.layers import Activation, Dropout, Flatten, Dense, GlobalMaxPooling2D, BatchNormalization, Input, Conv2D
from keras.callbacks import ModelCheckpoint
from keras import metrics
from keras.optimizers import Adam
from keras import backend as K
import keras
from keras.models import Model, Sequential
from keras.models import model_from_json
from keras import regularizers
from keras.losses import binary_crossentropy
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau
from keras.wrappers.scikit_learn import KerasClassifier

import tensorflow as tf

Using TensorFlow backend.


In [2]:
%matplotlib inline

In [3]:
start_time = time.time()

In [4]:
train_df = pd.read_csv('../input/train.csv')

In [5]:
test_df = pd.read_csv('../input/test.csv')

In [6]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Columns: 202 entries, ID_code to var_199
dtypes: float64(200), int64(1), object(1)
memory usage: 308.2+ MB


In [7]:
train_df.describe()

,target,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
count,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,...,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000
mean,0.100490,10.679914,-1.627622,10.715192,6.796529,11.078333,-5.065317,5.408949,16.545850,0.284162,...,3.234440,7.438408,1.927839,3.331774,17.993784,-0.142088,2.303335,8.908158,15.870720,-3.326537
std,0.300653,3.040051,4.050044,2.640894,2.043319,1.623150,7.863267,0.866607,3.418076,3.332634,...,4.559922,3.023272,1.478423,3.992030,3.135162,1.429372,5.454369,0.921625,3.010945,10.438015
min,0.000000,0.408400,-15.043400,2.117100,-0.040200,5.074800,-32.562600,2.347300,5.349700,-10.505500,...,-14.093300,-2.691700,-3.814500,-11.783400,8.694400,-5.261000,-14.209600,5.960600,6.299300,-38.852800
25%,0.000000,8.453850,-4.740025,8.722475,5.254075,9.883175,-11.200350,4.767700,13.943800,-2.317800,...,-0.058825,5.157400,0.889775,0.584600,15.629800,-1.170700,-1.946925,8.252800,13.829700,-11.208475
50%,0.000000,10.524750,-1.608050,10.580000,6.825000,11.108250,-4.833150,5.385100,16.456800,0.393700,...,3.203600,7.347750,1.901300,3.396350,17.957950,-0.172700,2.408900,8.888200,15.934050,-2.819550
75%,0.000000,12.758200,1.358625,12.516700,8.324100,12.261125,0.924800,6.003000,19.102900,2.937900,...,6.406200,9.512525,2.949500,6.205800,20.396525,0.829600,6.556725,9.593300,18.064725,4.836800
max,1.000000,20.315000,10.376800,19.353000,13.188300,16.671400,17.251600,8.447700,27.691800,10.151300,...,18.440900,16.716500,8.402400,18.281800,27.928800,4.272900,18.321500,12.000400,26.079100,28.500700


In [8]:
min_max_scaler = MinMaxScaler()

In [9]:
standart_scaler = StandardScaler()

In [10]:
robust_scaler = RobustScaler()

In [11]:
#scaled_train_values = min_max_scaler.fit_transform(train_df[train_df.columns.drop(['ID_code', 'target'])])
scaled_train_values = standart_scaler.fit_transform(train_df[train_df.columns.drop(['ID_code', 'target'])])
#scaled_train_values = robust_scaler.fit_transform(train_df[train_df.columns.drop(['ID_code', 'target'])])

In [12]:
target_values = train_df['target'].values

In [13]:
train_values, holdout_test_values, train_target_values, holdout_test_target_values = train_test_split(
    #scaled_train_values,
    train_df[train_df.columns.drop(['ID_code', 'target'])].values,
    target_values,
    test_size=0.2,
    random_state=0
)

In [14]:
print(train_values.shape)
print(train_target_values.shape)
print(holdout_test_values.shape)
print(holdout_test_target_values.shape)

(160000, 200)
(160000,)
(40000, 200)
(40000,)


In [15]:
#test_df.head()

In [16]:
ID_code = test_df['ID_code'].values

In [17]:
print(ID_code[:10])

['test_0' 'test_1' 'test_2' 'test_3' 'test_4' 'test_5' 'test_6' 'test_7'
 'test_8' 'test_9']


In [18]:
#tf.py_func

In [19]:
batch_size = 100

In [20]:
def auc(y_true, y_pred):
#def auc(y_pred, y_true):
    #print(y_true[:5])
    #print(y_pred[:5])  
    return tf.py_func(roc_auc_score, (y_true, y_pred), tf.double)
    #return tf.py_func(roc_auc_score, (y_true, y_pred), tf.int8)

Провести mean_shift кластеризацию на множестве переменных (features)?

In [21]:
#submission_predicts = sequential_nn_model.predict(test_df[test_df.columns.drop('ID_code')].values)

In [22]:
#submission_predicts.shape

In [23]:
#print(submission_predicts[:5])
#submission_predicts_values = submission_predicts[:, 0]

In [24]:
#submission_predicts_values.shape

In [25]:
#np.all(submission_predicts_values == 0)

In [26]:
#submission_df = pd.DataFrame({'ID_code': ID_code, 'target': submission_predicts_values.astype('float32')})

In [27]:
#np.any(submission_predicts_values == 1)

In [28]:
#np.all(submission_predicts_values > 0)

In [29]:
#np.all(submission_predicts_values < 0)

In [30]:
#np.any(submission_predicts_values > 0)

In [31]:
#np.any(submission_predicts_values < 0)

In [32]:
#submission_predicts_values[submission_predicts_values > 0].shape

In [33]:
#submission_predicts_values.max()

In [34]:
#submission_predicts_values.min()

In [35]:
#submission_predicts_values[submission_predicts_values < 0.245].shape

In [36]:
#submission_predicts_values[submission_predicts_values >= 0.245].shape

In [37]:
#submission_predicts_values[submission_predicts_values < 0.245] = 0
#submission_predicts_values[submission_predicts_values >= 0.245] = 1

In [38]:
#submission_predicts_values[submission_predicts_values == 1].shape

In [39]:
#submission_df = pd.DataFrame({'ID_code': ID_code, 'target': submission_predicts_values.astype('float32')})

In [40]:
#submission_df.to_csv('submission_mlp_1.csv', index=False)

In [73]:
def detect_threshold(classes_ratio, step_size, predicted_values):
    threshold = predicted_values.min()
    values_below_threshold = (predicted_values[predicted_values < threshold]).shape[0]
    values_above_threshold = (predicted_values[predicted_values >= threshold]).shape[0]
    #threshold_classes_ratio =  values_above_threshold / values_below_threshold
    threshold_classes_ratio = 1
    
    while(threshold_classes_ratio > classes_ratio):
        threshold += step_size        
        values_below_threshold = (predicted_values[predicted_values < threshold]).shape[0]
        values_above_threshold = (predicted_values[predicted_values >= threshold]).shape[0]
        threshold_classes_ratio =  values_above_threshold / values_below_threshold
    predicted_values[predicted_values < threshold] = 0
    predicted_values[predicted_values >= threshold] = 1
    return threshold, predicted_values

In [168]:
sequential_nn_model = None
#del sequential_nn_model
if sequential_nn_model:
    del sequential_nn_model
sequential_nn_model = Sequential()
sequential_nn_model.add(Dense(batch_size, input_dim=200, kernel_initializer='normal', activation='relu'))
sequential_nn_model.add(Dropout(0.1))
sequential_nn_model.add(Dense(batch_size, input_dim=100, kernel_initializer='normal', activation='sigmoid'))
sequential_nn_model.add(Dropout(0.1))
sequential_nn_model.add(Dense(batch_size, input_dim=100, kernel_initializer='normal', activation='relu'))
#sequential_nn_model.add(Dense(batch_size, input_shape=(100, 200), kernel_initializer='normal', activation='sigmoid'))
#sequential_nn_model.add(Dropout(0.76))
sequential_nn_model.add(Dropout(0.24))
sequential_nn_model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))

In [169]:
#sequential_nn_model.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.0005), metrics=['accuracy', auc])
sequential_nn_model.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.001), metrics=['accuracy', auc])

In [170]:
start_time = time.time()
sequential_nn_model.fit(train_values, train_target_values, batch_size=100, epochs=40, verbose=1, validation_split=0.2)
print("Run time {} min".format((time.time() - start_time) / 60))

Train on 128000 samples, validate on 32000 samples
Epoch 1/40
128000/128000 [==============================] - 7s 54us/step - loss: 0.3061 - acc: 0.8976 - auc: 0.7435 - val_loss: 0.2556 - val_acc: 0.9062 - val_auc: 0.8128
Epoch 2/40
128000/128000 [==============================] - 6s 46us/step - loss: 0.2642 - acc: 0.9034 - auc: 0.8146 - val_loss: 0.2476 - val_acc: 0.9098 - val_auc: 0.8251
Epoch 3/40
128000/128000 [==============================] - 6s 46us/step - loss: 0.2593 - acc: 0.9051 - auc: 0.8239 - val_loss: 0.2521 - val_acc: 0.9068 - val_auc: 0.8316
Epoch 4/40
128000/128000 [==============================] - 6s 47us/step - loss: 0.2564 - acc: 0.9063 - auc: 0.8291 - val_loss: 0.2456 - val_acc: 0.9107 - val_auc: 0.8343
Epoch 5/40
128000/128000 [==============================] - 6s 47us/step - loss: 0.2544 - acc: 0.9066 - auc: 0.8322 - val_loss: 0.2415 - val_acc: 0.9130 - val_auc: 0.8346
Epoch 6/40
128000/128000 [==============================] - 6s 47us/step - loss: 0.2535 - acc:

In [171]:
loss_and_metrics = sequential_nn_model.evaluate(holdout_test_values, holdout_test_target_values, batch_size=100)

40000/40000 [==============================] - 1s 21us/step


In [172]:
loss_and_metrics
# [0.25161404045298696, 0.9094000032544136, 0.8320361980522218] [0.2493363002128899, 0.9096750013530255, 0.8369347876192563]
# [0.24582509476691483, 0.9101250021159649, 0.8396344886385845] [0.24802235754206778, 0.9094000029563903, 0.8385308386618173]
# [0.24494799628853797, 0.9102000007033348, 0.839084512352511]

[0.24494799628853797, 0.9102000007033348, 0.839084512352511]

In [173]:
model_json = sequential_nn_model.to_json()
with open("sequential_nn_model_relu_droput024_lr001_sigmoid_40_epochs_2019-03-30-1.json", "w") as json_file:
    json_file.write(model_json)

In [174]:
sequential_nn_model.save("sequential_nn_model_relu_dropout024_lr001_sigmoid_40_epochs_2019-03-30-1.h5")

In [175]:
sequential_nn_model.save_weights("sequential_nn_model_weights_relu_dropout024_lr001_sigmoid_40_epochs_2019-03-30-1.h5")

In [176]:
submission_predict_values = sequential_nn_model.predict(test_df[test_df.columns.drop('ID_code')].values)[:, 0]

In [55]:
classes_ratio = train_df[train_df['target'] == 1].shape[0] / train_df[train_df['target'] == 0].shape[0]

In [56]:
classes_ratio

0.1117163789174106

In [177]:
threshold, submission_predicts = detect_threshold(classes_ratio, 0.01, submission_predict_values)

In [178]:
threshold

0.27001717647865014

In [179]:
submission_predicts[submission_predicts == 0].shape

(180136,)

In [180]:
submission_predicts[submission_predicts == 1].shape

(19864,)

In [181]:
submission_df = pd.DataFrame({'ID_code': ID_code, 'target': submission_predicts.astype('float32')})

In [182]:
submission_df.to_csv('submission_mlp_3.csv', index=False)